In [1]:
import numpy as np
import matplotlib.pyplot as plt
import vol
import scipy.stats as sps

from ipywidgets import interact
from ipywidgets import widgets
from tqdm.auto import tqdm

from dataclasses import dataclass
from typing import Union, Callable, Optional
from copy import deepcopy
from scipy.optimize import root_scalar, brentq
from dataclasses import dataclass

from scipy.interpolate import RectBivariateSpline

import warnings
from  scipy.stats import norm
warnings.filterwarnings("ignore")
from scipy.interpolate import CubicSpline
from tqdm import tqdm

#about super().__init__() https://habr.com/ru/company/skillfactory/blog/683744/

@dataclass
class StockOption: #общий класс для опционов
    strike_price: Union[float, np.ndarray]
    expiration_time: Union[float, np.ndarray]  # in years
    is_call: bool

@dataclass
class CallStockOption(StockOption): #класс для коллов
    def __init__(self, strike_price, expiration_time): 
        super().__init__(strike_price, expiration_time, True) #вызов метода __init__ из базового класса StockOption, т.е. вызываем колл     
        

@dataclass
class PutStockOption(StockOption):
    def __init__(self, strike_price, expiration_time):
        super().__init__(strike_price, expiration_time, False)
        
@dataclass
class MarketState:
    stock_price: Union[float, np.ndarray]
    interest_rate: Union[float, np.ndarray]  # r, assume constant

In [2]:
def dt(option: StockOption):
    return np.maximum(option.expiration_time, np.finfo(np.float64).eps)

def d1(option: StockOption, state: MarketState, vola: float):
    return 1 / (vola * np.sqrt(dt(option)))\
                * (np.log(state.stock_price / option.strike_price)
                   + (state.interest_rate + vola ** 2) * dt(option))

def d2(option: StockOption, state: MarketState, vola: float):
    return d1(option, state, vola) - vola * np.sqrt(dt(option))
    
def price(option: StockOption, state: MarketState, vola: float):
    discount_factor = np.exp(-state.interest_rate * (dt(option)))
    if option.is_call:
        return sps.norm.cdf(d1(option, state, vola)) * state.stock_price\
            - sps.norm.cdf(d2(option, state, vola)) * option.strike_price * discount_factor
    
    return sps.norm.cdf(-d2(option, state, vola)) * option.strike_price * discount_factor\
        - sps.norm.cdf(-d1(option, state, vola)) * state.stock_price

def vega(option: StockOption, state: MarketState, vola: float):
    return state.stock_price * sps.norm.pdf(d1(option, state, vola)) * np.sqrt(dt(option))

def calc_iv(option: CallStockOption, state: MarketState, option_price: float):
    
    RTOL = 1e-16
    ATOL = 1e-16
    VOL_MIN = 0.0
    VOL_MAX = 2.0
    MAX_ITER = 100
    
    vola = 0.2
    p = price(option, state, vola)

    lb = VOL_MIN
    rb = VOL_MAX
    
    for i in range(MAX_ITER):
        
        resid = price(option, state, vola) - option_price
        if abs(resid) < ATOL and abs(resid) / (option_price + 1e-4) < RTOL:
            break
        
        newton_ok = False
        
        # try Newton
        v = vega(option, state, vola)
        if abs(v) > 1e-4:
            new_vola = vola - (p - option_price) / v
            if lb < new_vola < rb:
                newton_ok = True
        
        # if something went wrong, bisect
        if not newton_ok:
            new_vola = (lb + rb) / 2

        new_p = price(option, state, new_vola)
        if new_p > option_price:
            rb = new_vola
        else:
            lb = new_vola
        
        vola = new_vola
        p = new_p
        
    return vola

In [3]:
MyBSM=vol.BlackScholes(s=100,sigma=0.2, r=0)
Ks=np.linspace(70,130,10)
Ts=np.linspace(0,10,31)
MyCallPrices=MyBSM.call_price(Ts.reshape(1,-1),Ks.reshape(-1,1))
MyCallPrices.shape

(10, 31)

In [4]:
observed_calls = CallStockOption(strike_price=Ks,expiration_time=Ts)
observed_call_prices = MyCallPrices
observed_market_state = MarketState(stock_price=100,interest_rate=0.0)

In [5]:
IV = np.empty_like(observed_call_prices) #market implied vol
for i in range(len(observed_calls.strike_price)):
    for j in range(len(observed_calls.expiration_time)):
        IV[i, j] = calc_iv(option=CallStockOption(observed_calls.strike_price[i], observed_calls.expiration_time[j]), 
                           state=observed_market_state, 
                           option_price=observed_call_prices[i, j])
IV.shape

(10, 31)

In [8]:
stock_prices=MyBSM.simulate(10,30,100000)
stock_prices=stock_prices.mean(axis=1)
stock_prices

array([100.        , 100.04387757, 100.03430073, 100.04044798,
       100.02564047, 100.00744123,  99.9861155 ,  99.96272674,
        99.97427945,  99.93188344,  99.89735471,  99.91748898,
        99.91574106,  99.97421843,  99.96589625,  99.87476409,
        99.84752665,  99.81303291,  99.89807546,  99.87445009,
        99.9269306 ,  99.86653086,  99.83229198,  99.90898779,
        99.9622983 ,  99.91126295,  99.8614626 ,  99.92014079,
        99.96107294,  99.87686565,  99.86566051])

In [14]:
S0=100
K=110
T=3
iv_sigm=IV[6][9]
init_call_price=MyCallPrices[6][9]
print("Price=",init_call_price,"iv_sigm=",iv_sigm)

Price= 9.975687261637248 iv_sigm= 0.20322549239855542


In [32]:
cash=init_call_price
amt_stocks=0
for t,st in zip(Ts[0:9],stock_prices[0:9]):
    d1=(np.log(st/K)+iv_sigm*iv_sigm*0.5*(T-t))/(iv_sigm*np.sqrt(T-t))
    delta=-sps.norm.cdf(d1)
    print(t,st,cash,amt_stocks,delta)
    amt_stocks+=delta
    cash-=delta*st
print(cash,amt_stocks)
print(cash+amt_stocks*stock_prices[9])
print(stock_prices[9])

0.0 100.0 9.975687261637248 0 -0.4622481549762299
0.3333333333333333 100.04387757077559 56.20050275926024 -0.4622481549762299 -0.45226528651649933
0.6666666666666666 100.03430073282696 101.44687571302865 -0.9145134414927292 -0.44010480861831713
1.0 100.04044797568422 145.47245249231665 -1.3546182501110464 -0.4260204887151662
1.3333333333333333 100.02564047067467 188.0917330302018 -1.7806387388262126 -0.40861181327307694
1.6666666666666665 100.00744122769615 228.96339135672503 -2.1892505520992893 -0.3864798894971511
2.0 99.98611549609367 267.61425619129784 -2.5757304415964404 -0.35641505627050973
2.333333333333333 99.96272674010413 303.25081317210777 -2.93214549786695 -0.31076960542678245
2.6666666666666665 99.9742794469895 334.3161903185152 -3.2429151032937327 -0.22487372868217814
356.7977793100738 -3.467788831975911
10.2551099688921
99.93188343700997
